In [ ]:
# these two things reimport modules if they change (needed for changing eval_functions)
%reload_ext autoreload
%autoreload 2

In [ ]:
import eval_functions
from eval_functions import *
import pandas as pd

# make tables interactive
from itables import init_notebook_mode
import itables.options as opt
init_notebook_mode(all_interactive=True, connected=True)
opt.maxBytes=0

In [ ]:
NOODLER=""
CVC5="cvc5-1.1.2"
Z3="z3-4.13.0"
Z3STR4="z3str4"
OSTRICH="ostrich-5dd2e10ca"
Z3STR3RE="z3strRE"
Z3TRAU="z3-trau-1.1"
OLD_NOODLER="z3-noodler-ec4ce36-d95fe13"

TOOLS = [
    NOODLER,
    OLD_NOODLER,
    CVC5,
    Z3,
    # Z3STR4,
    OSTRICH,
    # Z3STR3RE,
    # Z3TRAU,
]

VBS = [
    [Z3, CVC5],
    [NOODLER, Z3, CVC5],
]

restrict_to_int_conversion = True

if not restrict_to_int_conversion:
  BENCHES = [
      "sygus_qgen",
      "denghang",
      "automatark",
      "stringfuzz",

      "norn",
      "slog",
      "slent",
      "transducer_plus",
      "kepler",
      "woorpje",
      "webapp",
      "kaluza",
      "redos",

      "leetcode",
      "str_small_rw",
      "pyex",
      "full_str_int",
      ]
else:
  # Only benchmarks with to_int/from_int
  BENCHES = [
      "stringfuzz",
      "str_small_rw",
      "full_str_int",
  ]

In [ ]:
dfs = dict()
for bench in BENCHES:
  df = read_file(bench + "/to120.csv")
  df["benchmark"] = bench
  dfs[bench] = df

# we select only columns with used tools
df_all = pd.concat(dfs, ignore_index=True)[["benchmark"] + ["name"] + [f(tool) for tool in TOOLS for f in (lambda x: x+"-result", lambda x: x+"-runtime")]]

for tool in TOOLS:
  # set runtime to 120 for nonsolved instances (unknown, TO, ERR or something else)
  df_all.loc[(df_all[f"{tool}-result"] != " sat")&(df_all[f"{tool}-result"] != " unsat"), f"{tool}-runtime"] = 120
  # runtime columns should be floats
  df_all[f"{tool}-runtime"] = df_all[f"{tool}-runtime"].astype(float)

if restrict_to_int_conversion:
  # we select only those formulae that contain to_int/from_int
  with open("int_convs_not-full_str_int.txt") as file:
    # fsi_not_conv is a list of formulae from full_str_int that do not contain to_int/from_int
    fsi_not_conv = file.read().splitlines()
  with open("int_convs-str_small_rw.txt") as file:
    # ssr_conv is a list of formulae from str_small_rw that contain to_int/from_int
    ssr_conv = file.read().splitlines()
  with open("int_convs-stringfuzz.txt") as file:
    # sf_conv is a list of formulae from stringfuzz that contain to_int/from_int
    sf_conv = file.read().splitlines()
  df_all = df_all[(df_all.benchmark != "full_str_int/to120.csv")|(~(df_all.name.isin(fsi_not_conv)))]
  df_all = df_all[((df_all.benchmark != "str_small_rw/to120.csv")&(df_all.benchmark != "stringfuzz/to120.csv"))|((df_all.name.isin(ssr_conv))|(df_all.name.isin(sf_conv)))]

# TODO VBS are ugly for now, will fix it
for vbs in VBS:
  name = "+".join(vbs)
  df_all = add_vbs(df_all, vbs, name)
  TOOLS.append(name)

### Evaluation

In [ ]:
print(simple_table(df_all, TOOLS, BENCHES,
                   separately=True # comment this to print one table for all benchmarks together
))

print(cactus_plot(df_all, TOOLS, start=int(len(df_all)*0.9)))

for tool in TOOLS:
    if tool != NOODLER:
        print(scatter_plot(df_all, NOODLER, tool))


### More detailed evaluation

In [ ]:
# check if noodler does not return different result than other solvers (i.e. wrong sat/unsat)
sanity_check(df_all, NOODLER, [tool for tool in TOOLS if tool!=NOODLER])

In [ ]:
# get all formulae where noodler gives different result than sat/unsat/unknown/TO
res = get_errors(df_all, NOODLER)

# filter out "ERR", usually those are MEMOUTs
# res = res[(res[NOODLER + '-result'] != "ERR")]

res

In [ ]:
get_timeouts(df_all, NOODLER)

In [ ]:
get_unknowns(df_all, NOODLER)

In [ ]:
get_solved(df_all, NOODLER)

### TODO: For papers (tables and figures with nicer names or something) 

In [ ]:
tool_names_mapping = {
    NOODLER : "Z3-Noodler",
    CVC5 : "cvc5",
    Z3 : "Z3",
    Z3STR4 : "Z3stsr4",
    OSTRICH : "OSTRICH",
    Z3STR3RE : "Z3str3RE",
    Z3TRAU : "Z3-Trau",
    OLD_NOODLER : "Z3-Noodler*",
}

tool_latex_mapping = {
    NOODLER : "\\ziiinoodler",
    CVC5 : "\\cvcv",
    Z3 : "\\ziii",
    Z3STR4 : "\\ziiistriv",
    OSTRICH : "\\ostrich",
    Z3STR3RE : "\\ziiistriiire",
    Z3TRAU : "\\ziiitrau",
    OLD_NOODLER : "\\ziiinoodlerold",
}

# TODO add table generation for latex